# Linear Regression

In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [2]:
class Linear_Regression():
    def __init__(self, alpha=1e-10 , num_iter=10000, early_stop=1e-50, intercept=True, init_weight=None): 
        """
        Some initializations, if neccesary
        
        attributes: 
            alpha: Learning Rate, default 1e-10
            num_iter: Number of Iterations to update coefficient with training data
            early_stop: Constant control early_stop.
            intercept: Bool, If we are going to fit a intercept, default True.
            init_weight: Matrix (n x 1), input init_weight for testing.
                    
        TODO: 1. Initialize all variables needed.
        """
        self.model_name = 'Linear Regression'
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight
    
    def fit(self, X_train, y_train):
        """
        Save the datasets in our model, and perform gradient descent.
        
        Parameter:
            X_train: Matrix or 2-D array. Input feature matrix.
            Y_train: Matrix or 2-D array. Input target value.
            
        TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
              3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
              4. Call the gradient_descent function to train.
        """
        self.X = np.asmatrix(X_train)
        self.y = np.asmatrix(y_train).T
        self.num_samples, self.num_features = self.X.shape
        self.coef = self.init_weight
        
        if self.intercept:
            ones = np.ones(shape=(self.X.shape[0], 1))
            self.X = np.hstack((ones, X))
            self.num_features += 1

        if self.init_weight is None:
            self.coef = np.random.uniform(-1, 1, (self.num_features, 1))

        self.gradient_descent()
        
    def gradient(self):
        """
        Helper function to calculate the gradient respect to coefficient.
        
        TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        y_pred = np.dot(self.X, self.coef)
        error = self.y - y_pred
        self.grad_coef = (-2 / len(self.X)) * self.X.T * error
        
    def gradient_descent(self):
        """
        Training function
        
        TODO: 6. Calculate the loss with current coefficients.
              7. Update the temp_coef with learning rate and gradient.
              8. Calculate the loss with temp_coef.
              9. Implement the self adeptive learning rate. 
                  a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                     And update coef, with temp_coef.
                  b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                     Don't update coef.
              10. Add the loss to loss list we create.
        """
        self.loss = []
        y_pred = np.dot(self.X, self.coef)
        pre_error = float(np.sum(np.power(self.y - y_pred, 2)) / len(self.X))
        self.loss.append(pre_error)
        
        for i in range(self.num_iter):                
            self.gradient()

            temp_coef = self.coef - self.alpha * self.grad_coef
            self.coef, old_coef = temp_coef, self.coef.copy()
            y_pred = np.dot(self.X, self.coef)
            current_error = float(np.sum(np.power(self.y - y_pred, 2)) / len(self.X))
            
            # This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                self.alpha *= 1.3
            else:
                self.alpha *= 0.9
                self.coef = old_coef
                current_error = pre_error
                
            self.loss.append(current_error)
            pre_error = current_error

            # Print weights and loss information
            # if i % 10000 == 0:
            #     print('Iteration: ' +  str(i))
            #     print('Coef: '+ str(self.coef))
            #     print('Loss: ' + str(current_error))            
        return self
    
    def ind_predict(self, x: list):
        """
        Predict the value based on its feature vector x.

        Parameter:
        x: Matrix, array or list. Input feature point.
        
        Return:
            result: prediction of given data point

        TODO: 11. Implement the prediction function
        """
        x = np.asmatrix(x)       
        result = float(x * self.coef)
        return result
    
    def predict(self, X):
        """
        X is a matrix or 2-D numpy array, represnting testing instances. 
        Each testing instance is a feature vector. 
        
        Parameter:
        X: Matrix, array or list. Input feature point.
        
        Return:
            ret: prediction of given data matrix
  
        TODO: 12. Make sure add the 1's column like we did to add intercept.
              13. Revise the following for-loop to call ind_predict to get predictions.
        """
        ret = []
        X = np.asmatrix(X)
        if self.intercept:
            ones = np.ones(shape=(X.shape[0], 1))
            X = np.hstack((ones, X))
            
        for i in range(X.shape[0]):
            ret.append(self.ind_predict(X[i]))
        return ret

In [3]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [4]:
X = np.array(np.asmatrix(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() + 20

Do **NOT** modify the following line, just run it when you are done. You can also try different initialization, you will notice different coef at the end.

In [5]:
clf = Linear_Regression(alpha=1, num_iter=10000000, init_weight=np.asmatrix([15,25]).T)
clf.fit(X,y)

As the number of iteration increase, you should notice the coeficient converges to [20, 30]. It maybe very slow update. Feel free to stop.

In [6]:
clf.coef

matrix([[19.93498063],
        [30.00009746]])

In [7]:
np.array(clf.predict(X))

/var/folders/pr/b_ytjflj1fqffyn22t_b8x_m0000gn/T/ipykernel_3386/1245635744.py:121: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = float(x * self.coef)


array([   49.93507809,   199.93556539,   349.93605269,   499.93653999,
         649.93702729,   799.9375146 ,   949.9380019 ,  1099.9384892 ,
        1249.9389765 ,  1399.9394638 ,  1549.9399511 ,  1699.94043841,
        1849.94092571,  1999.94141301,  2149.94190031,  2299.94238761,
        2449.94287492,  2599.94336222,  2749.94384952,  2899.94433682,
        3049.94482412,  3199.94531142,  3349.94579873,  3499.94628603,
        3649.94677333,  3799.94726063,  3949.94774793,  4099.94823524,
        4249.94872254,  4399.94920984,  4549.94969714,  4699.95018444,
        4849.95067174,  4999.95115905,  5149.95164635,  5299.95213365,
        5449.95262095,  5599.95310825,  5749.95359556,  5899.95408286,
        6049.95457016,  6199.95505746,  6349.95554476,  6499.95603206,
        6649.95651937,  6799.95700667,  6949.95749397,  7099.95798127,
        7249.95846857,  7399.95895588,  7549.95944318,  7699.95993048,
        7849.96041778,  7999.96090508,  8149.96139238,  8299.96187969,
      

Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [8]:
X_norm, y_norm = min_max_normaliz(X), min_max_normaliz(y)
clf.fit(X_norm,y_norm)

In [9]:
clf.coef

matrix([[0.19386068],
        [0.00071316]])

In [10]:
np.array(clf.predict(X))

/var/folders/pr/b_ytjflj1fqffyn22t_b8x_m0000gn/T/ipykernel_3386/1245635744.py:121: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = float(x * self.coef)


array([0.19457384, 0.19813963, 0.20170543, 0.20527122, 0.20883701,
       0.2124028 , 0.21596859, 0.21953439, 0.22310018, 0.22666597,
       0.23023176, 0.23379755, 0.23736334, 0.24092914, 0.24449493,
       0.24806072, 0.25162651, 0.2551923 , 0.25875809, 0.26232389,
       0.26588968, 0.26945547, 0.27302126, 0.27658705, 0.28015285,
       0.28371864, 0.28728443, 0.29085022, 0.29441601, 0.2979818 ,
       0.3015476 , 0.30511339, 0.30867918, 0.31224497, 0.31581076,
       0.31937655, 0.32294235, 0.32650814, 0.33007393, 0.33363972,
       0.33720551, 0.34077131, 0.3443371 , 0.34790289, 0.35146868,
       0.35503447, 0.35860026, 0.36216606, 0.36573185, 0.36929764,
       0.37286343, 0.37642922, 0.37999501, 0.38356081, 0.3871266 ,
       0.39069239, 0.39425818, 0.39782397, 0.40138977, 0.40495556,
       0.40852135, 0.41208714, 0.41565293, 0.41921872, 0.42278452,
       0.42635031, 0.4299161 , 0.43348189, 0.43704768, 0.44061347,
       0.44417927, 0.44774506, 0.45131085, 0.45487664, 0.45844

You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density', 'alcohol']]
y = wine.quality

In [13]:
lr = LinearRegression()
lr.fit(X,y)

# Squared Error with sklearn
print(f'Coefficients: {lr.coef_}')
print(f'Squared Error: {sum((lr.predict(X) - y)**2)}')

Coefficients: [34.82170159  0.39144139]
Squared Error: 800.6676988774332


You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [14]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X, y)

In [15]:
print(f'Squared Error: {sum((clf.predict(X) - y)**2)}')

Squared Error: 805.3513209405593


/var/folders/pr/b_ytjflj1fqffyn22t_b8x_m0000gn/T/ipykernel_3386/1245635744.py:121: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = float(x * self.coef)
